# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 19:36:39] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36405, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=99976

[2025-08-03 19:36:39] Using default HuggingFace chat template with detected content format: string


[2025-08-03 19:36:47] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 19:36:47] Init torch distributed begin.


[2025-08-03 19:36:49] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 19:36:50] Load weight begin. avail mem=78.58 GB


...

[2025-08-03 19:36:55] Init torch distributed begin.
[2025-08-03 19:36:55] Init torch distributed ends. mem usage=0.00 GB
[2025-08-03 19:36:55] Load weight begin. avail mem=55.14 GB
[2025-08-03 19:36:55] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.19s/it]

[2025-08-03 19:36:56] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=53.72 GB, mem usage=1.42 GB.
[2025-08-03 19:36:56] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-03 19:36:56] Memory pool end. avail mem=53.41 GB


[2025-08-03 19:36:56] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=53.63 GB
[2025-08-03 19:36:56] INFO:     Started server process [3076633]
[2025-08-03 19:36:56] INFO:     Waiting for application startup.
[2025-08-03 19:36:56] INFO:     Application startup complete.
[2025-08-03 19:36:56] INFO:     Uvicorn running on http://127.0.0.1:36405 (Press CTRL+C to quit)


[2025-08-03 19:36:57] INFO:     127.0.0.1:37786 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 19:36:57] INFO:     127.0.0.1:37790 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 19:36:57] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 19:38:03] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-03 19:38:33] INFO:     127.0.0.1:37798 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 19:38:33] The server is fired up and ready to roll!


[2025-08-03 19:38:36] INFO:     127.0.0.1:56652 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 19:38:45] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32321, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=5407251

[2025-08-03 19:38:46] Using default HuggingFace chat template with detected content format: string


[2025-08-03 19:38:55] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 19:38:55] Init torch distributed begin.


[2025-08-03 19:38:56] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 19:38:57] Load weight begin. avail mem=61.44 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]

[2025-08-03 19:39:05] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=8.51 GB, mem usage=21.66 GB.
[2025-08-03 19:39:05] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-03 19:39:05] Memory pool end. avail mem=8.19 GB


[2025-08-03 19:39:06] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=8.29 GB
[2025-08-03 19:39:06] INFO:     Started server process [3083841]
[2025-08-03 19:39:06] INFO:     Waiting for application startup.
[2025-08-03 19:39:06] INFO:     Application startup complete.
[2025-08-03 19:39:06] INFO:     Uvicorn running on http://127.0.0.1:32321 (Press CTRL+C to quit)


[2025-08-03 19:39:06] INFO:     127.0.0.1:58292 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 19:39:07] INFO:     127.0.0.1:58298 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 19:39:07] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:39:10] INFO:     127.0.0.1:58300 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 19:39:10] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 19:39:11] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:39:12] INFO:     127.0.0.1:51820 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 19:39:20] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32036, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-03 19:39:20] Using default HuggingFace chat template with detected content format: string


[2025-08-03 19:39:27] Casting torch.bfloat16 to torch.float16.


[2025-08-03 19:39:27] Casting torch.bfloat16 to torch.float16.
[2025-08-03 19:39:27] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 19:39:27] Init torch distributed begin.


[2025-08-03 19:39:28] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.08s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.08s/it]

[2025-08-03 19:39:48] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=42.17 GB, mem usage=1.70 GB.
[2025-08-03 19:39:48] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-08-03 19:39:48] Memory pool end. avail mem=42.09 GB


[2025-08-03 19:39:49] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.76 GB


[2025-08-03 19:39:49] INFO:     Started server process [3086534]
[2025-08-03 19:39:49] INFO:     Waiting for application startup.
[2025-08-03 19:39:49] INFO:     Application startup complete.
[2025-08-03 19:39:49] INFO:     Uvicorn running on http://127.0.0.1:32036 (Press CTRL+C to quit)
[2025-08-03 19:39:49] INFO:     127.0.0.1:55282 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 19:39:50] INFO:     127.0.0.1:55286 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 19:39:50] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:39:53] INFO:     127.0.0.1:55292 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 19:39:53] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 19:39:54] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:39:55] INFO:     127.0.0.1:55300 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 19:40:02] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35862, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-03 19:40:03] Using default HuggingFace chat template with detected content format: string


[2025-08-03 19:40:10] Casting torch.bfloat16 to torch.float16.


[2025-08-03 19:40:10] Casting torch.bfloat16 to torch.float16.
[2025-08-03 19:40:10] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 19:40:10] Init torch distributed begin.


[2025-08-03 19:40:11] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 19:40:29] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=40.22 GB


[2025-08-03 19:40:30] INFO:     Started server process [3088893]
[2025-08-03 19:40:30] INFO:     Waiting for application startup.
[2025-08-03 19:40:30] INFO:     Application startup complete.
[2025-08-03 19:40:30] INFO:     Uvicorn running on http://127.0.0.1:35862 (Press CTRL+C to quit)


[2025-08-03 19:40:30] INFO:     127.0.0.1:41844 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 19:40:31] INFO:     127.0.0.1:41858 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 19:40:31] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:40:34] INFO:     127.0.0.1:41868 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 19:40:34] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 19:40:35] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:40:35] INFO:     127.0.0.1:41870 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 19:40:42] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30695, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1034319138, constrained_js

[2025-08-03 19:40:43] Using default HuggingFace chat template with detected content format: string


[2025-08-03 19:40:50] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 19:40:50] Init torch distributed begin.


[2025-08-03 19:40:51] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 19:40:52] Load weight begin. avail mem=78.58 GB


...

[2025-08-03 19:40:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=59.84 GB


[2025-08-03 19:40:58] INFO:     Started server process [3090865]
[2025-08-03 19:40:58] INFO:     Waiting for application startup.
[2025-08-03 19:40:58] INFO:     Application startup complete.
[2025-08-03 19:40:58] INFO:     Uvicorn running on http://0.0.0.0:30695 (Press CTRL+C to quit)


[2025-08-03 19:40:59] INFO:     127.0.0.1:60076 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 19:40:59] INFO:     127.0.0.1:60088 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 19:40:59] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:41:01] INFO:     127.0.0.1:60090 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 19:41:01] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-03 19:41:04] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 19:41:04] Decode batch. #running-req: 1, #token: 93, token usage: 0.00, accept len: 1.81, cuda graph: False, gen throughput (token/s): 11.65, #queue-req: 0, 


[2025-08-03 19:41:05] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 102.74, #queue-req: 0, 


[2025-08-03 19:41:05] Decode batch. #running-req: 1, #token: 226, token usage: 0.01, accept len: 1.62, cuda graph: False, gen throughput (token/s): 100.20, #queue-req: 0, 


[2025-08-03 19:41:06] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, accept len: 1.65, cuda graph: False, gen throughput (token/s): 101.82, #queue-req: 0, 


[2025-08-03 19:41:07] Decode batch. #running-req: 1, #token: 362, token usage: 0.02, accept len: 1.75, cuda graph: False, gen throughput (token/s): 107.77, #queue-req: 0, 


[2025-08-03 19:41:07] INFO:     127.0.0.1:60104 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).